In [1]:
import numpy as np
import itertools
import warnings
warnings.filterwarnings('ignore')
from inputs import texts
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity



In [2]:
def generate_candidates(ngram_range,stop_words,doc):
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
    candidates = count.get_feature_names()
    return candidates

def get_embeddings(model,doc,candidates):
    doc_embedding = model.encode([doc])
    candidate_embedding = model.encode(candidates)
    
    return doc_embedding,candidate_embedding

def get_keywords(doc_embedding,candidate_embedding,topk=5):
    
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-topk:]]
    
    return keywords

def main(doc,ngram_range,stop_words,model,topk=5):
    
    candidates = generate_candidates(ngram_range,stop_words,doc)
    
    doc_embed,candidate_embed = get_embeddings(model,doc,candidates)
    
    keywords = get_keywords(doc_embed,candidate_embed,topk)
    
    return keywords
    
    
main(doc,n_gram_range,stop_words,model)
    

NameError: name 'doc' is not defined

In [3]:
#candiadtate keyword
n_gram_range = (1, 1)

stop_words = "english"

doc= texts[-1]
# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names()

In [4]:
candidates

['19',
 '380',
 '40',
 '73',
 '78',
 'according',
 'american',
 'appear',
 'arab',
 'association',
 'backing',
 'bahrain',
 'beijing',
 'biological',
 'carried',
 'china',
 'claimed',
 'company',
 'concluded',
 'countries',
 'covid',
 'data',
 'details',
 'developed',
 'east',
 'effective',
 'egypt',
 'emergency',
 'emirates',
 'health',
 'institute',
 'involved',
 'jordan',
 'journal',
 'just',
 'local',
 'medical',
 'middle',
 'online',
 'organization',
 'participants',
 'partners',
 'placebo',
 'previously',
 'products',
 'provided',
 'published',
 'recently',
 'report',
 'researchers',
 'safe',
 'say',
 'scientists',
 'sinopharm',
 'study',
 'trial',
 'uae',
 'united',
 'use',
 'used',
 'vaccines',
 'waiting',
 'won',
 'world',
 'wuhan']

In [5]:
#Embeddings
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [6]:
#cosine similarity
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [7]:
indices = distances.argsort()

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
for index in distances.argsort()[0][-top_n:][::-1]:
    print(candidates[index],distances[0][index])

In [ ]:
#diversification
import numpy as np
import itertools

def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [33]:
def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
% matplotlib inline

In [9]:
import streamlit.components.v1

from htbuilder import H, HtmlElement, styles
from htbuilder.units import unit

# Only works in 3.7+: from htbuilder import div, span
div = H.div
span = H.span

# Only works in 3.7+: from htbuilder.units import px, rem, em
px = unit.px
rem = unit.rem
em = unit.em


def annotation(body, label="", background="#ddd", color="#333", **style):
    """Build an HtmlElement span object with the given body and annotation label.
    The end result will look something like this:
        [body | label]
    Parameters
    ----------
    body : string
        The string to put in the "body" part of the annotation.
    label : string
        The string to put in the "label" part of the annotation.
    background : string
        The color to use for the background "chip" containing this annotation.
    color : string
        The color to use for the body and label text.
    **style : dict
        Any CSS you want to use to customize the containing "chip".
    Examples
    --------
    Produce a simple annotation with default colors:
    >>> annotation("apple", "fruit")
    Produce an annotation with custom colors:
    >>> annotation("apple", "fruit", background="#FF0", color="black")
    Produce an annotation with crazy CSS:
    >>> annotation("apple", "fruit", background="#FF0", border="1px dashed red")
    """

    if "font_family" not in style:
        style["font_family"] = "sans-serif"

    return span(
        style=styles(
            background=background,
            border_radius=rem(0.33),
            color=color,
            padding=(rem(0.17), rem(0.67)),
            display="inline-flex",
            justify_content="center",
            align_items="center",
            **style,
        )
    )(
        body,
        span(
            style=styles(
                color=color,
                font_size=em(0.67),
                opacity=0.5,
                padding_left=rem(0.5),
                text_transform="uppercase",
                margin_bottom=px(-2),
            )
        )(label)
    )


ModuleNotFoundError: No module named 'htbuilder'